# 라이브러리 및 경로 설정

In [1]:
import pandas as pd

In [2]:
src_path = '/content/drive/MyDrive/sk쉴더스/프로젝트1/데이터/'

# 데이터 로드

In [38]:
grids = pd.read_csv(src_path + 'df_grid')
grids = grids.iloc[:, 1:]
grids.shape

(3227, 5)

In [37]:
# gid가 중복되는 행을 없앤 데이터프레임
# 단순히 없앴기 때문에 여러구에 걸쳤다는 정보도 사라짐
# 이 데이터프레임에서 중복되는 격자의 '구' 값을 수정하면 중복 처리 완료
unique_grids = grids.drop_duplicates(['gid'])
unique_grids.shape

(2634, 5)

# gid 중복 처리

```python
key 값이 중복되는 격자 이름, value 값이 빈 리스트인 딕셔너리를 만들고 value(리스트)에 '구'를 추가할 예정
예를 들어 '다사1234'인 격자가 '마포구', '강남구'에 걸쳐있다면
dup_grid_gu['다사1234'].append('마포구')
dup_grid_gu['다사1234'].append('강남구')
위의 두 코드를 통해 딕셔너리를
{ '다사1234' : [] }
에서
{ '다사1234' : ['마포구', '강남구'] }
로 만든다.

그리고 이 딕셔너리를 통해 데이터프레임(unique_grids)의 '구' 값을 바꿔줄 예정
```

In [30]:
# 중복 격자 이름 리스트
dup_grids = grids[grids['gid'].duplicated()]['gid'].unique()

In [31]:
dup_grid_gu = {k : [] for k in dup_grids}

In [32]:
# 기존에 중복이 있는 grids 데이터프레임 중 gid가 dup_grids(중복 격자 이름 리스트)에 들어가는 
# 행들을 돌면서 해당 격자의 구를 추가한다 
for row in grids[[gid in dup_grids for gid in grids['gid']]].iterrows():
  gid = row[1]['gid']
  gu = row[1]['gu']
  
  dup_grid_gu[gid].append(gu)

In [41]:
dup_grid_gu

{'다사65a49a': ['강동구', '광진구', '송파구'],
 '다사65b49b': ['강동구', '광진구'],
 '다사65b50b': ['강동구', '광진구'],
 '다사65b50a': ['강동구', '광진구'],
 '다사65b49a': ['강동구', '광진구', '송파구'],
 '다사61a47a': ['강남구', '광진구'],
 '다사61a47b': ['강남구', '광진구'],
 '다사66a50b': ['강동구', '광진구'],
 '다사60b47b': ['강남구', '광진구', '성동구'],
 '다사61b47a': ['강남구', '광진구', '송파구'],
 '다사46b42b': ['관악구', '구로구'],
 '다사46b42a': ['관악구', '구로구', '금천구'],
 '다사47a42b': ['관악구', '구로구'],
 '다사47a43a': ['관악구', '구로구', '동작구', '영등포구'],
 '다사47b42b': ['관악구', '금천구'],
 '다사47a42a': ['관악구', '금천구'],
 '다사48b39b': ['관악구', '금천구'],
 '다사48a40b': ['관악구', '금천구'],
 '다사44b43a': ['구로구', '금천구'],
 '다사47b41b': ['관악구', '금천구'],
 '다사45a42b': ['구로구', '금천구'],
 '다사47b41a': ['관악구', '금천구'],
 '다사48a40a': ['관악구', '금천구'],
 '다사49a39a': ['관악구', '금처

In [49]:
# 위에서 중복되는 행을 단순 제거했기 때문에 여러 구에 걸쳐있는 격자인데 gu 정보가 하나밖에 없음
# 이제 이 값을 딕셔너리에 넣었던 값으로 대체하면 될 것 같음
cnt = 0
for k, v in dup_grid_gu.items():
  cnt += 1
  print(k, unique_grids[unique_grids['gid']==k]['gu'].values, ' -> ', v)
  if(cnt == 5):
    break

다사65a49a ['강동구']  ->  ['강동구', '광진구', '송파구']
다사65b49b ['강동구']  ->  ['강동구', '광진구']
다사65b50b ['강동구']  ->  ['강동구', '광진구']
다사65b50a ['강동구']  ->  ['강동구', '광진구']
다사65b49a ['강동구']  ->  ['강동구', '광진구', '송파구']


In [64]:
# 리스트의 값들을 공백을 기준으로 하나의 문자열로 합침
# ['강동구', '광진구'] -> '강동구 광진구'
# 그 후 값으로 넣어줌
for k, v in dup_grid_gu.items():
  unique_grids.loc[unique_grids['gid']==k, 'gu'] = ' '.join(v)

In [65]:
# 잘 들어갔는지 확인
unique_grids[[gid in dup_grids for gid in unique_grids['gid']]]

,gid,lbl,val,geometry,gu
5,다사64b43b,3628.0,3628.0,"POLYGON ((964500 1943500, 964500 1944000, 9650...",강남구 송파구
15,다사61b46b,689.0,689.0,"POLYGON ((961500 1946500, 961500 1947000, 9620...",강남구 송파구
16,다사63b40b,609.0,609.0,"POLYGON ((963500 1940500, 963500 1941000, 9640...",강남구 서초구
21,다사57b46a,3315.0,3315.0,"POLYGON ((957500 1946000, 957500 1946500, 9580...",강남구 서초구
30,다사60a47b,3862.0,3862.0,"POLYGON ((960000 1947500, 960000 1948000, 9605...",강남구 성동구
...,...,...,...,...,...
2994,다사53b52a,26.0,26.0,"POLYGON ((953500 1952000, 953500 1952500, 9540...",종로구 중구
2995,다사56b52a,358.0,358.0,"POLYGON ((956500 1952000, 956500 1952500, 9570...",종로구 중구
3009,다사55b52a,85.0,85.0,"POLYGON ((955500 1952000, 955500 1952500, 9560...",종로구 중구
3011,다사56a52a,87.0,87.0,"POLYGON ((956000 1952000, 956000 1952500, 9565...",종로구 중구


In [68]:
# 인덱스가 0부터 고르게 있지 않아서 고르게 만들어줌
unique_grids.reset_index(drop=True)

,gid,lbl,val,geometry,gu
0,다사58a48a,6085.0,6085.0,"POLYGON ((958000 1948000, 958000 1948500, 9585...",강남구
1,다사58b48a,3446.0,3446.0,"POLYGON ((958500 1948000, 958500 1948500, 9590...",강남구
2,다사58a45b,7734.0,7734.0,"POLYGON ((958000 1945500, 958000 1946000, 9585...",강남구
3,다사61b44a,10261.0,10261.0,"POLYGON ((961500 1944000, 961500 1944500, 9620...",강남구
4,다사58a47b,4755.0,4755.0,"POLYGON ((958000 1947500, 958000 1948000, 9585...",강남구
...,...,...,...,...,...
2629,다사66a54b,NaN,NaN,"POLYGON ((966000 1954500, 966000 1955000, 9665...",중랑구
2630,다사66a56b,NaN,NaN,"POLYGON ((966000 1956500, 966000 1957000, 9665...",중랑구
2631,다사65b55a,NaN,NaN,"POLYGON ((965500 1955000, 965500 1955500, 9660...",중랑구
2632,다사66a57a,NaN,NaN,"POLYGON ((966000 1957000, 966000 1957500, 9665...",중랑구


In [69]:
unique_grids.to_csv(src_path + 'unique_grid.csv', index=False)

In [70]:
pd.read_csv(src_path + 'unique_grid.csv')

,gid,lbl,val,geometry,gu
0,다사58a48a,6085.0,6085.0,"POLYGON ((958000 1948000, 958000 1948500, 9585...",강남구
1,다사58b48a,3446.0,3446.0,"POLYGON ((958500 1948000, 958500 1948500, 9590...",강남구
2,다사58a45b,7734.0,7734.0,"POLYGON ((958000 1945500, 958000 1946000, 9585...",강남구
3,다사61b44a,10261.0,10261.0,"POLYGON ((961500 1944000, 961500 1944500, 9620...",강남구
4,다사58a47b,4755.0,4755.0,"POLYGON ((958000 1947500, 958000 1948000, 9585...",강남구
...,...,...,...,...,...
2629,다사66a54b,NaN,NaN,"POLYGON ((966000 1954500, 966000 1955000, 9665...",중랑구
2630,다사66a56b,NaN,NaN,"POLYGON ((966000 1956500, 966000 1957000, 9665...",중랑구
2631,다사65b55a,NaN,NaN,"POLYGON ((965500 1955000, 965500 1955500, 9660...",중랑구
2632,다사66a57a,NaN,NaN,"POLYGON ((966000 1957000, 966000 1957500, 9665...",중랑구
